In [13]:
import requests
from bs4 import BeautifulSoup
from time import sleep
from tqdm import tqdm
import pandas as pd
import numpy as np
import re
import json
from gensim.utils import deaccent

unable to import 'smart_open.gcs', disabling that module


In [120]:
class Downloader():
    def __init__(self):
        self.link = 'https://www.bezrealitky.cz/vypis/nabidka-pronajem/byt/praha?_token=pr1lf-vKwDFfmFbICiz2PfC-Zdwq-2JolXi4MeMHsrw&page=1'
        self.request = requests.get(self.link)
        self.request.encoding='UTF-8'
        self.soup = BeautifulSoup(self.request.text,'lxml')
        
    def get_data(self):
        descrips = []
        values = []
        vals = self.soup.findAll('strong', {'class':'product__value'})
        for vl in vals:
            values.append(vl.text.strip())
        img = self.soup.findAll('img')
        for i in img:
            if 'Pronájem' and 'obr. č. 1' in i['alt']:
                    info = i['alt'].split(',')[0:4]
                    if 'Praha' == info[-1].strip():
                        info.insert(2, np.NaN)
                        del info[-1]
                        descrips.append(info)
                    else:
                        descrips.append(info)
        print(descrips)
        count = 0
        for pp in values:
            descrips[count].append(pp)
            descrips[count][0] = descrips[count][0][-4:].strip()
            count += 1
        print(descrips)
        for item in descrips:
            prices = item.pop(4).split('+')
            item.append(re.sub("[^0-9]", "", prices[0]))
            item.append(re.sub("[^0-9]", "", prices[1]))
        dicts = {}
        count = 0
        for item in descrips:
            dict = {}
            dict['Size'] = item[0]
            dict['m2'] = re.sub("[^0-9]", "", item[1])
            dict['Street'] = deaccent(item[2])
            dict['District'] = deaccent(item[3])
            dict['Base Price'] = int(item[4])
            dict['Utilities Price'] = int(item[5])
            dict['Total Price'] = int(item[4]) + int(item[5])
            dicts[count] = dict
            count += 1
        with open('bezrealitky.json', 'w') as write_file:
            json.dump(dicts, write_file, indent = 4)
        print(descrips)
        
        
            
        

In [119]:
a = Downloader()
a.get_data()

[['Pronájem bytu 2+kk', ' 57 m²', ' Novovysočanská', ' Praha - Vysočany'], ['Pronájem bytu 3+kk', ' 89 m²', ' Sanderova Street', ' Praha - Holešovice'], ['Pronájem bytu 2+kk', ' 45 m²', nan, ' Praha - Žižkov'], ['Pronájem bytu 2+kk', ' 62 m²', ' Sanderova', ' Praha - Holešovice'], ['Pronájem bytu 2+kk', ' 83 m²', ' Nad Ostrovem', ' Praha - Podolí'], ['Pronájem bytu 1+kk', ' 29 m²', ' Sokolovská', ' Praha - Karlín'], ['Pronájem bytu 1+1', ' 40 m²', ' Práčská', ' Praha - Záběhlice'], ['Pronájem bytu 3+kk', ' 57 m²', ' Krakovská', ' Praha - Nové Město'], ['Pronájem bytu 2+kk', ' 54 m²', ' Rohanské Nábřeží', ' Praha - Karlín'], ['Pronájem bytu 1+1', ' 35 m²', ' Radimova', ' Praha - Břevnov']]
[['2+kk', ' 57 m²', ' Novovysočanská', ' Praha - Vysočany', '15.990 Kč + 2.800 Kč'], ['3+kk', ' 89 m²', ' Sanderova Street', ' Praha - Holešovice', '33.000 Kč + 6.750 Kč'], ['2+kk', ' 45 m²', nan, ' Praha - Žižkov', '13.000 Kč + 1.300 Kč'], ['2+kk', ' 62 m²', ' Sanderova', ' Praha - Holešovice', '27.0

AttributeError: 'float' object has no attribute 'decode'